Выведите в нормальном виде результат обучения. А лучше график для accuracy.
Еще попробуйте логистическую регрессию.

Чтобы не было проблем с памятью, вот такого стоит избегать: Variable(torch.from_numpy(batch)).float().cuda().

Создайте одну переменную и меняйте ее содержимое.

In [79]:
import torch
import torchvision
from torchvision.models import alexnet
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
import numpy as np
from torch.autograd import Variable
from scipy.misc import imresize
import torch.cuda
from tqdm import tqdm
from sklearn.linear_model import LinearRegression, LogisticRegression

 Задание: выкинуть последний слой из vgg / alexnet и обучить линейную регрессиию на выходах полученных сетей.
Например, для mnist.

In [20]:
anet = torchvision.models.alexnet(pretrained = True)
fmodel = anet.features

In [21]:
for m in fmodel.children():
    for p in m.parameters():
        p.requires_grad = False

In [22]:
fmodel.cuda()

Sequential (
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU (inplace)
  (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU (inplace)
  (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU (inplace)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU (inplace)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU (inplace)
  (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
)

In [23]:
mnist = fetch_mldata('MNIST original', data_home='~/.datasets')
images = mnist['data']
images = images.reshape(images.shape[0], 1, 28, 28) # data dims: examples x channels x w x h
images = images.astype(np.float32) / 255 # float and normalize
labels = mnist['target'].astype(int)
np_images_train, np_images_test, np_labels_train, np_labels_test = train_test_split(images, labels, train_size = 0.9)

In [24]:
def resize(batch):
    factor = 8
    resized_images = np.empty((batch.shape[0], 1, batch.shape[2] * 8, batch.shape[3] * 8))
    for i in xrange(batch.shape[0]):
        resized_images[i, 0, :] = imresize(batch[i, 0], size = float(factor))
    out_images = np.concatenate([resized_images] * 3, axis = 1)
    return out_images

In [25]:
def learn(data, labels, iters = None):
    if iters is None:
        iters = int(np.ceil(float(data.shape[0]) / batchsize))
    batchsize = 64
    res = np.empty((batchsize * iters, 256 * 6 * 6))
    res_labels = labels[:batchsize * iters]
    invar = Variable(torch.Tensor(), requires_grad = False)
    for it in tqdm(xrange(iters)):
        orig_batch = data[it * batchsize:(it + 1) * batchsize]
        batch = resize(orig_batch)
        ten = torch.from_numpy(batch).float().cuda()
        invar.data = ten
        outvar = fmodel.forward(invar)
        flat = outvar.view(outvar.size(0), -1)
        flatres = flat.data.cpu().numpy()
        res[it * batchsize:(it + 1) * batchsize] = flatres
    return res, res_labels

In [32]:
def learn(data, labels, iters = None):
    if iters is None:
        iters = int(np.ceil(float(data.shape[0]) / batchsize))
    batchsize = 64
    res = np.empty((batchsize * iters, 256 * 6 * 6))
    res_labels = labels[:batchsize * iters]
    for it in tqdm(xrange(iters)):
        orig_batch = data[it * batchsize:(it + 1) * batchsize]
        batch = resize(orig_batch)
        invar = Variable(torch.from_numpy(batch)).float().cuda()
        outvar = fmodel.forward(invar)
        flat = outvar.view(outvar.size(0), -1)
        flatres = flat.data.cpu().numpy()
        res[it * batchsize:(it + 1) * batchsize] = flatres
    return res, res_labels

In [ ]:
test_iters = 20
test_res, test_res_labels = learn(np_images_test, np_labels_test, test_iters)

In [ ]:
np.save('res', res)
np.save('labels', res_labels)

In [ ]:
res = np.load('res.npy')
res_labels = np.load('labels.npy')

In [85]:
logr = LogisticRegression()

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [91]:
logr.fit(res, res_labels)
np.save('logr_coef', logr.coef_)
np.save('logr_inter', logr.intercept_)
np.save('logr_niter', logr.n_iter_)

In [101]:
logr.coef_ = np.load('logr_coef.npy')
logr.intercept_ = np.load('logr_inter.npy')
logr.n_iter_ = np.load('logr_niter.npy')

In [102]:
logr_pred = logr.predict(test_res)

In [112]:
diff = np.abs(logr_pred - test_res_labels)

In [105]:
print('% of right answers: ' + str(float(diff[diff < 0.5].shape[0]) / diff.shape[0]))

% of right answers: 0.9828125


In [ ]:
lin = LinearRegression()

In [ ]:
lin.fit(res, res_labels)
np.save('lin_coef', lin.coef_)
np.save('lin_inter', lin.intercept_)

In [ ]:
# for ind in xrange(5):
#     i = 10 - ind * 2
#     cur_res = res[:res.shape[0] / i, :]
#     cur_res_labels = res_labels[:res_labels.shape[0] / i]
#     lin.fit(cur_res, cur_res_labels)
#     np.save('coef' + str(i), lin.coef_)
#     print(i)

In [ ]:
lin.coef_ = np.load('lin_coef.npy')
lin.intercept_ = np.load('lin_inter.npy')

In [106]:
lin_pred = lin.predict(test_res)

In [107]:
diff = np.abs(lin_pred - test_res_labels)

In [108]:
print('% of right answers: ' + str(float(diff[diff < 0.5].shape[0]) / diff.shape[0]))

% of right answers: 0.19296875


In [111]:
# linear regression has weird coefficient spikes:
print(lin.coef_.mean())
np.median(lin.coef_)

2487813.08242


0.0

In [ ]:
# buf it still performs slightly better than random